In [236]:
import os
import os.path as ospath
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import glob

from picasso.picasso import io


In [13]:
def load_ring_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("No results of previously analyzed datasets were detected.")
        return None
    else: 
        print("Results of previously analyzed datasets were detected.")
        return df
    
def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")

        
    return locs, info, pixelsize

In [124]:
path = r'W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\FtsZ\FtsZ_picks_all'



df_ring_data = load_ring_data_df(path, "ring_data.pkl")
print(df_ring_data.keys())

# radius criterium
dR = 100
perc_on_ring_min = 0.7
perc_inside_ring_max = 0.2
perc_outside_ring_max = 0.2

# angle criterium
N_segments = 8
perc_locs_per_filled_segment_min = 1/N_segments/4 # a segment counts as filled if it contains this fraction of all localizations
perc_segments_filled_min = 0.5 # this fraction of segments has to be filled to accept the ring



Results of previously analyzed datasets were detected.
Index(['fov_id', 'cell_type', 'filename', 'group', 'residual', 'radius',
       'angle_between_ring_and_coverslip', 'Sigma', 'n_events', 'mean_bright',
       'mean_dark', 'fit_bright', 'fit_dark'],
      dtype='object')


['W:\\users\\reinhardt\\z.software\\Git\\spor-PAINT\\dev_sr\\spor-paint\\FtsZ\\FtsZ_picks_all\\analysis\\excluded_filter\\fov_6_spor_pick_3.png', 'W:\\users\\reinhardt\\z.software\\Git\\spor-PAINT\\dev_sr\\spor-paint\\FtsZ\\FtsZ_picks_all\\analysis\\excluded_filter\\fov_8_spor_pick_14.png']
hello
W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\FtsZ\FtsZ_picks_all\analysis\excluded_filter\fov_6_spor_pick_3.png
hello
fov_6_spor_pick_3.png


In [ ]:
analysis_folder = os.path.join(path, 'analysis')
excluded_folder = os.path.join(path, 'analysis', 'excluded_filter')
if not os.path.isdir(excluded_folder):
    os.makedirs(excluded_folder)

# reset excluded_filter folder

# move fov_1_spor_pick_0.png back to analysis folder
print(glob.glob(os.path.join(excluded_folder, '*.png')))
for image_path in glob.glob(os.path.join(excluded_folder, '*.png')):
    if 'filter.png' in image:
        os.remove(image_path)
    else:
        image = os.path.split(image_path)[1]
        shutil.move(os.path.join(excluded_folder, image), os.path.join(analysis_folder, image))
       #shutil.move(os.path.join(analysis_folder, filename_base + '.png'), os.path.join(excluded_folder, filename_base + '.png'))


orientation = 'yz' # use yz, but also xy files contain info for filtering



# prepare bins for angle check:
# angles from -180 to 180
bin_width = 360/N_segments
bins = np.arange(-180, 180+bin_width, bin_width)


# add filter_passed column:
df_ring_data['filter_passed'] = np.full(len(df_ring_data), "No")
df_ring_data['filename_ring'] =  'fov_' + df_ring_data['fov_id'].astype(str) +'_' + df_ring_data['cell_type'].astype(str) +'_pick_' + df_ring_data['group'].astype(str) +'_ring_' + '0' +'_rot_' + orientation + '_update.hdf5'
for index, ring in df_ring_data.iterrows():
    plt.close('all')
    locs, info, pixelsize = load_data(os.path.join(path, 'analysis', 'ring_locs', ring['filename_ring'] ))
    
    radius_fit = ring['radius']
    radius_min = radius_fit - dR
    radius_max = radius_fit + dR
    
    
    N_locs = len(locs)

    ################
    # Check if radius criterium is fulfilled.
    ################
    
    perc_on_ring = len(locs[(locs['radius']>radius_min) & (locs['radius']<radius_max)]) / N_locs
    perc_inside_ring = len(locs[locs['radius']<radius_min]) / N_locs
    perc_outside_ring = len(locs[locs['radius']>radius_max]) / N_locs
    
    radius_criterium = perc_on_ring >= perc_on_ring_min and perc_inside_ring <= perc_inside_ring_max and perc_outside_ring <= perc_outside_ring_max
    
    #print(perc_on_ring, perc_on_ring >= perc_on_ring_min)
    #print(perc_inside_ring, perc_inside_ring <= perc_inside_ring_max)
    #print(perc_outside_ring, perc_outside_ring <= perc_outside_ring_max)
    #print(radius_criterium, radius_criterium)
    #print()
    
    ################
    # Check if angle criterium is fulfilled.
    ################
    print(ring['filename_ring'])
    df_locs = pd.DataFrame.from_records(locs)
    perc_locs_per_segment = df_locs['angle'].groupby(pd.cut(df_locs['angle'], bins = bins)).count()/N_locs
    print(perc_locs_per_segment)
    #print(perc_locs_per_filled_segment_min)
    perc_locs_per_segments_filled = perc_locs_per_segment[perc_locs_per_segment>perc_locs_per_filled_segment_min]
    perc_segments_filled = len(perc_locs_per_segments_filled)/N_segments
    print('perc_locs_per_segments_filled', perc_locs_per_segments_filled)
    #print('perc_segments_filled', perc_segments_filled)
    #print('perc_segments_filled_min', perc_segments_filled_min)
    
    angle_criterium = perc_segments_filled >= perc_segments_filled_min
    #print(angle_criterium)
    #print()
    


    ################
    # Plot
    ################    
    
    # set up plot illustrating analysis parameters
    fig = plt.figure(figsize=(20, 10), constrained_layout=True)
    gs = fig.add_gridspec(1,2)
    fig.suptitle(("FOV {}, {}, Pick {} - Ring analysis\n"
                  "File: {}\n").format(ring['fov_id'], ring['cell_type'] , ring['group'], ring['filename']), 
                 fontsize=16, 
                 ha="center")

    # set maximum radius for plot such that 99% of locs are shown
    cutoff = 1

    radius_sort = np.sort(locs['radius'])
    radius_limit = radius_sort[int(cutoff*len(radius_sort))-1]

    ## ring criterium
    ax_1 = fig.add_subplot(gs[0, 0], projection='polar')
    
    ax_1.set_xticks(bins[1:]/180*np.pi)
    ax_1.set_thetalim(-np.pi,np.pi)

    #ring of fit radius
    coords_angle_fit = np.arange(-np.pi,np.pi,0.01)
    coords_radius_fit = np.full(len(coords_angle_fit), radius_fit)
    coords_radius_min = np.full(len(coords_angle_fit), radius_min)
    coords_radius_max = np.full(len(coords_angle_fit), radius_max)
    
    ax_1.fill_between(coords_angle_fit,radius_min, radius_max, color='red', alpha=0.2)

    ax_1.plot(coords_angle_fit, coords_radius_fit, c='red', lw=2)
    ax_1.plot(coords_angle_fit, coords_radius_min, c='red', lw=1)
    ax_1.plot(coords_angle_fit, coords_radius_max, c='red', lw=1)
    
    # locs
    ax_1.scatter(locs['angle']/180*np.pi, locs['radius'], alpha=0.5)
    
    
    # set maximum radius for plot such that 99% of locs are shown
    ax_1.set_rmax(radius_limit)

    ax_1.set_title('Localizations within ring\n\n\n\n\n', fontsize = 16)
    
    annotate_y = 0.85
    plt.annotate("Fit radius: {:.1f} nm, defined ring width: $\pm${:.0f} nm\n".expandtabs().format(radius_fit,dR),xy=(0.05,annotate_y-0.02), xycoords = 'subfigure fraction',fontsize=14)
    plt.annotate("Percent of locs on ring: {:.1f}% \t\t Min. percent of locs on ring: {:.1f}%\n".expandtabs().format(perc_on_ring*100,perc_on_ring_min*100),xy=(0.05,annotate_y-0.04), xycoords = 'subfigure fraction',fontsize=14)
    plt.annotate("Percent of locs inside of ring: {:.1f}%  \t Max. percent of locs inside of ring: {:.1f}%\n".expandtabs().format(perc_inside_ring*100,perc_inside_ring_max*100),xy=(0.05,annotate_y-0.06), xycoords = 'subfigure fraction',fontsize=14)
    plt.annotate("Percent of locs outside of ring: {:.1f}% \t Max. percent of locs outside of ring: {:.1f}%\n".expandtabs().format(perc_outside_ring*100,perc_outside_ring_max*100),xy=(0.05,annotate_y-0.08), xycoords = 'subfigure fraction',fontsize=14)

    

    
    ## angle criterium
    ax_2 = fig.add_subplot(gs[0, 1], projection='polar')
    
    # segments
    ax_2.set_xticks(bins[1:]/180*np.pi)
    ax_2.set_thetalim(-np.pi,np.pi)
    #ax_2.tick_params(axis='x', width = 5, color = 'red')
    ax_2.xaxis.grid(linewidth=2, color = 'red')

    # filled segments
    for segment in perc_locs_per_segments_filled.index:
        coords_angle_segment = np.arange(segment.left/180*np.pi,segment.right/180*np.pi,0.01)
        ax_2.fill_between(coords_angle_segment,0, radius_limit, color='red', alpha=0.2)
       
    #ax_2.annotate(text = 'Hello', xy = (np.pi,500))
    for segment in perc_locs_per_segment.index:
        dangle = segment.right-segment.left
        angle_annotate = (segment.right-dangle/2)/180*np.pi
        ax_2.annotate(text = "{:.1f}%".format(perc_locs_per_segment[segment]*100),xy=(angle_annotate,radius_limit-50), ha='center', va='center')

    # locs
    ax_2.scatter(locs['angle']/180*np.pi, locs['radius'], alpha=0.5)

    # set maximum radius for plot such that 99% of locs are shown
    ax_2.set_rmax(radius_limit)
    
    ax_2.set_rlabel_position(2)

    ax_2.set_title('Localizations across {:.0f} segments\n\n\n\n\n'.format(N_segments), fontsize = 16)
    
    plt.annotate("Percentage of segments with more than {:.1f}% of locs: {:.1f}% ".expandtabs().format(perc_locs_per_filled_segment_min*100,perc_segments_filled*100),xy=(0.55,annotate_y-0.02), xycoords = 'subfigure fraction',fontsize=14)
    plt.annotate("Min. percentage of segments with more than {:.1f}% of locs: {:.1f}% ".expandtabs().format(perc_locs_per_filled_segment_min*100, perc_segments_filled_min*100),xy=(0.55,annotate_y-0.04), xycoords = 'subfigure fraction',fontsize=14)

    ################
    # Save result of criteria check in ring_data dataframe
    ################

        
    filename_base = 'fov_' + str(ring['fov_id']) +'_' + ring['cell_type'] +'_pick_' + str(ring['group'])
    filename_result =  filename_base +'_filter.png'
    if radius_criterium and angle_criterium:
        df_ring_data.loc[index,'filter_passed'] = 'Yes'
        fig.savefig(os.path.join(analysis_folder, filename_result), dpi=300, format="png")

    else:
        df_ring_data.loc[index,'filter_passed'] = 'No'
        fig.savefig(os.path.join(excluded_folder, filename_result), dpi=300, format="png")
        
        # Move fov_1_spor_pick_0.png file excluded_filter
        
        shutil.move(os.path.join(analysis_folder, filename_base + '.png'), os.path.join(excluded_folder, filename_base + '.png'))

        
df_ring_data = df_ring_data.drop(columns='filename_ring')
#df_ring_data['filename_ring']


"""
4. plot
6. Move image in analysis folder to excluded.
Save filtering metrics
"""





[]
fov_1_veg_pick_0_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.207226
(-135.0, -90.0]     0.079755
(-90.0, -45.0]      0.104294
(-45.0, 0.0]        0.171779
(0.0, 45.0]         0.166326
(45.0, 90.0]        0.075665
(90.0, 135.0]       0.074301
(135.0, 180.0]      0.120654
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.207226
(-135.0, -90.0]     0.079755
(-90.0, -45.0]      0.104294
(-45.0, 0.0]        0.171779
(0.0, 45.0]         0.166326
(45.0, 90.0]        0.075665
(90.0, 135.0]       0.074301
(135.0, 180.0]      0.120654
Name: angle, dtype: float64
fov_1_veg_pick_1_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.188347
(-135.0, -90.0]     0.032520
(-90.0, -45.0]      0.140921
(-45.0, 0.0]        0.174797
(0.0, 45.0]         0.135501
(45.0, 90.0]        0.105691
(90.0, 135.0]       0.109756
(135.0, 180.0]      0.112466
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.188347
(-135.0, -90.0]   

fov_3_veg_pick_1_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.112429
(-135.0, -90.0]     0.015819
(-90.0, -45.0]      0.080226
(-45.0, 0.0]        0.064972
(0.0, 45.0]         0.270621
(45.0, 90.0]        0.122599
(90.0, 135.0]       0.068927
(135.0, 180.0]      0.264407
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.112429
(-90.0, -45.0]      0.080226
(-45.0, 0.0]        0.064972
(0.0, 45.0]         0.270621
(45.0, 90.0]        0.122599
(90.0, 135.0]       0.068927
(135.0, 180.0]      0.264407
Name: angle, dtype: float64
fov_3_veg_pick_2_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.233074
(-135.0, -90.0]     0.179149
(-90.0, -45.0]      0.235470
(-45.0, 0.0]        0.179748
(0.0, 45.0]         0.052127
(45.0, 90.0]        0.019772
(90.0, 135.0]       0.014979
(135.0, 180.0]      0.085680
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.233074
(-135.0, -90.0]     0.179149
(-90.0, -45.0]      0

fov_6_spor_pick_5_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.177515
(-135.0, -90.0]     0.039448
(-90.0, -45.0]      0.049310
(-45.0, 0.0]        0.000000
(0.0, 45.0]         0.301775
(45.0, 90.0]        0.027613
(90.0, 135.0]       0.130178
(135.0, 180.0]      0.274162
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.177515
(-135.0, -90.0]     0.039448
(-90.0, -45.0]      0.049310
(0.0, 45.0]         0.301775
(90.0, 135.0]       0.130178
(135.0, 180.0]      0.274162
Name: angle, dtype: float64
fov_7_veg_pick_0_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.100692
(-135.0, -90.0]     0.160478
(-90.0, -45.0]      0.127753
(-45.0, 0.0]        0.138452
(0.0, 45.0]         0.101951
(45.0, 90.0]        0.123348
(90.0, 135.0]       0.133417
(135.0, 180.0]      0.113908
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.100692
(-135.0, -90.0]     0.160478
(-90.0, -45.0]      0.127753
(-45.0, 0.0]        

fov_7_veg_pick_13_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.094203
(-135.0, -90.0]     0.169887
(-90.0, -45.0]      0.187601
(-45.0, 0.0]        0.123188
(0.0, 45.0]         0.125604
(45.0, 90.0]        0.132045
(90.0, 135.0]       0.102254
(135.0, 180.0]      0.065217
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.094203
(-135.0, -90.0]     0.169887
(-90.0, -45.0]      0.187601
(-45.0, 0.0]        0.123188
(0.0, 45.0]         0.125604
(45.0, 90.0]        0.132045
(90.0, 135.0]       0.102254
(135.0, 180.0]      0.065217
Name: angle, dtype: float64
fov_7_veg_pick_14_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.086149
(-135.0, -90.0]     0.108953
(-90.0, -45.0]      0.193412
(-45.0, 0.0]        0.043074
(0.0, 45.0]         0.190878
(45.0, 90.0]        0.133446
(90.0, 135.0]       0.093750
(135.0, 180.0]      0.150338
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.086149
(-135.0, -90.0]    

fov_8_spor_pick_7_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.173840
(-135.0, -90.0]     0.168776
(-90.0, -45.0]      0.073418
(-45.0, 0.0]        0.006751
(0.0, 45.0]         0.037131
(45.0, 90.0]        0.209283
(90.0, 135.0]       0.126582
(135.0, 180.0]      0.204219
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.173840
(-135.0, -90.0]     0.168776
(-90.0, -45.0]      0.073418
(0.0, 45.0]         0.037131
(45.0, 90.0]        0.209283
(90.0, 135.0]       0.126582
(135.0, 180.0]      0.204219
Name: angle, dtype: float64
fov_8_spor_pick_8_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.047297
(-135.0, -90.0]     0.152510
(-90.0, -45.0]      0.203668
(-45.0, 0.0]        0.078185
(0.0, 45.0]         0.063707
(45.0, 90.0]        0.015444
(90.0, 135.0]       0.082046
(135.0, 180.0]      0.357143
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.047297
(-135.0, -90.0]     0.152510
(-90.0, -45.0]     

...

fov_2_spor_pick_9_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.187929
(-135.0, -90.0]     0.077275
(-90.0, -45.0]      0.135802
(-45.0, 0.0]        0.172840
(0.0, 45.0]         0.178784
(45.0, 90.0]        0.117513
(90.0, 135.0]       0.071788
(135.0, 180.0]      0.058070
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.187929
(-135.0, -90.0]     0.077275
(-90.0, -45.0]      0.135802
(-45.0, 0.0]        0.172840
(0.0, 45.0]         0.178784
(45.0, 90.0]        0.117513
(90.0, 135.0]       0.071788
(135.0, 180.0]      0.058070
Name: angle, dtype: float64
fov_2_spor_pick_10_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.103051
(-135.0, -90.0]     0.193220
(-90.0, -45.0]      0.076610
(-45.0, 0.0]        0.147797
(0.0, 45.0]         0.123390
(45.0, 90.0]        0.121356
(90.0, 135.0]       0.138983
(135.0, 180.0]      0.095593
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.103051
(-135.0, -90.0]   

fov_1_spor_pick_1_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.115385
(-135.0, -90.0]     0.186235
(-90.0, -45.0]      0.070850
(-45.0, 0.0]        0.141700
(0.0, 45.0]         0.097166
(45.0, 90.0]        0.119433
(90.0, 135.0]       0.117409
(135.0, 180.0]      0.151822
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.115385
(-135.0, -90.0]     0.186235
(-90.0, -45.0]      0.070850
(-45.0, 0.0]        0.141700
(0.0, 45.0]         0.097166
(45.0, 90.0]        0.119433
(90.0, 135.0]       0.117409
(135.0, 180.0]      0.151822
Name: angle, dtype: float64
fov_1_spor_pick_2_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.131510
(-135.0, -90.0]     0.148438
(-90.0, -45.0]      0.182292
(-45.0, 0.0]        0.067708
(0.0, 45.0]         0.134115
(45.0, 90.0]        0.130208
(90.0, 135.0]       0.067708
(135.0, 180.0]      0.138021
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.131510
(-135.0, -90.0]    

fov_19_spor_pick_5_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.123239
(-135.0, -90.0]     0.140845
(-90.0, -45.0]      0.080986
(-45.0, 0.0]        0.102113
(0.0, 45.0]         0.297535
(45.0, 90.0]        0.021127
(90.0, 135.0]       0.070423
(135.0, 180.0]      0.163732
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.123239
(-135.0, -90.0]     0.140845
(-90.0, -45.0]      0.080986
(-45.0, 0.0]        0.102113
(0.0, 45.0]         0.297535
(90.0, 135.0]       0.070423
(135.0, 180.0]      0.163732
Name: angle, dtype: float64
fov_19_spor_pick_6_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.162272
(-135.0, -90.0]     0.085193
(-90.0, -45.0]      0.099391
(-45.0, 0.0]        0.068966
(0.0, 45.0]         0.196755
(45.0, 90.0]        0.089249
(90.0, 135.0]       0.070994
(135.0, 180.0]      0.227181
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.162272
(-135.0, -90.0]     0.085193
(-90.0, -45.0]   

fov_21_spor_pick_0_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.275463
(-135.0, -90.0]     0.078704
(-90.0, -45.0]      0.074074
(-45.0, 0.0]        0.096065
(0.0, 45.0]         0.163194
(45.0, 90.0]        0.010417
(90.0, 135.0]       0.034722
(135.0, 180.0]      0.267361
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.275463
(-135.0, -90.0]     0.078704
(-90.0, -45.0]      0.074074
(-45.0, 0.0]        0.096065
(0.0, 45.0]         0.163194
(90.0, 135.0]       0.034722
(135.0, 180.0]      0.267361
Name: angle, dtype: float64
fov_21_spor_pick_1_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.233553
(-135.0, -90.0]     0.008224
(-90.0, -45.0]      0.003289
(-45.0, 0.0]        0.009868
(0.0, 45.0]         0.013158
(45.0, 90.0]        0.093750
(90.0, 135.0]       0.409539
(135.0, 180.0]      0.228618
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.233553
(45.0, 90.0]        0.093750
(90.0, 135.0]    

fov_24_spor_pick_1_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.136288
(-135.0, -90.0]     0.128532
(-90.0, -45.0]      0.230471
(-45.0, 0.0]        0.127978
(0.0, 45.0]         0.134072
(45.0, 90.0]        0.055956
(90.0, 135.0]       0.036011
(135.0, 180.0]      0.150693
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.136288
(-135.0, -90.0]     0.128532
(-90.0, -45.0]      0.230471
(-45.0, 0.0]        0.127978
(0.0, 45.0]         0.134072
(45.0, 90.0]        0.055956
(90.0, 135.0]       0.036011
(135.0, 180.0]      0.150693
Name: angle, dtype: float64
fov_24_spor_pick_2_ring_0_rot_yz_update.hdf5
angle
(-180.0, -135.0]    0.173600
(-135.0, -90.0]     0.173272
(-90.0, -45.0]      0.148379
(-45.0, 0.0]        0.073698
(0.0, 45.0]         0.155912
(45.0, 90.0]        0.061251
(90.0, 135.0]       0.079921
(135.0, 180.0]      0.133967
Name: angle, dtype: float64
perc_locs_per_segments_filled angle
(-180.0, -135.0]    0.173600
(-135.0, -90.0]  

In [ ]:
df_ring_data.to_csv(os.path.join(path,"ring_data_filter.csv"))
# Save dataframe with cell means for easy loading of data for postprocessing
df_ring_data.to_pickle(os.path.join(path,"ring_data_filter.pkl"))